# Kickstart Seed Project - Web Scraper for Audio Files


## How to use this notebook

I've set up this notebook to hopefully be as easy to follow as possible, and barring the file paths for saving extracted files, little else should need modification.

Throughout this notebook, cells or lines that require customizing will be marked in `code blocks` in text cells above the code in question. Further documentation and explanation of the code itself is commented within each code block.

As I wrote this notebook in Google Colab, I've begun the notebook with the extra step of mounting Google Drive to connect it to the notebook. This is unnecessary if you are running the notebook locally instead.

In [ ]:
# run if mounting google drive
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
!pip install xlsxwriter

In [ ]:
import pandas as pd
import os
import re
import json

## Download youtube videos using Youtube_DL
Documentation: https://github.com/yt-dlp/yt-dlp

Using a fork of Youtube_dl, yt-dlp, to complete this section since Youtube_DL no longer works after a change in Youtube's metadata in 2021.

In [ ]:
!pip install yt_dlp

In [ ]:
from yt_dlp import YoutubeDL

In [ ]:
# define a logger object to catch/print error messages
class Logger(object):
  def debug(self, msg):
    pass

  def warning(self, msg):
    pass

  def error(self, msg):
    print(msg)

# define a hook to keep track of download status
def hook(d):
    if d['status'] == 'finished':
      print('Download complete, now converting...')

Three changes need to be made in this code block:
1. Under `options`, there is a key for `'outtmpl'` which specifies the output template as defined [here](https://github.com/yt-dlp/yt-dlp?tab=readme-ov-file#output-template). It is currently set to automatically name extracted videos with the format "ID.extension" (where ID is the last part of the video's url and extension is mp4). Change the file path from everything before `/%(id)s.%(ext)s` to where you want the downloaded files to go. As a side note, you can change the `format` to `mp3/bestaudio/best` and it should just extract the mp3 file, but please refer to [yt-dlp's documentation](https://github.com/yt-dlp/yt-dlp) for details.
2. For the `playlist` variable, simply paste the link to the Youtube playlist and the code will take care of the rest. Note that the library can only download 100 videos at once, so break the playlist up if you are extracting more than 100 videos (there is code to take care of this problem later on). In the event that there are more than 100 videos, when you are extracting the second playlist make sure to set `i` to `101` or the relevant number as this variable defines the streamlined rename of the videos for easier reference. The current convention is language_number.extension, for example, `en_1.mp4`.
3. To change the language in the file name, it is in the `row.append("fr_" + str(i)` line. Remember to change this when switching to a different language, or if a different file naming convention is established.

Here are a few playlist links that I used for testing and for extracting our final dataset:
* English test playlist: https://youtube.com/playlist?list=PLGeLaOAVnnkt8ZwHdTwto_eXtgERr4THd&si=sFsnQneXhatZLoNV
* French test playlist: https://youtube.com/playlist?list=PLX6CyFIhOy_wk645GTARAkbIqhdqAms2o&si=PKORJWBYPBSLNlQ7
* English masterlist: https://youtube.com/playlist?list=PLGeLaOAVnnkuuUfW3E-4RFDely0fvODDx&si=2PjQtZhfBUhuMz2t
* Second half of English masterlist: https://youtube.com/playlist?list=PLGeLaOAVnnktvT5uxUZukZ4zeorWL-jUW&si=RdIqO7uAJANcTw2o
* French masterlist: https://youtube.com/playlist?list=PLX6CyFIhOy_z7cO9hBB8RhSw42CRI5jTe&si=bcdricJWnnIl8gRo

In [ ]:
# specify options for downloaded files
options = {
    'format': 'mp4/bestvideo/best',
    'logger': Logger(),
    'progress_hooks': [hook],
    'outtmpl': '/content/drive/My Drive/kickstarter_seed_project/fr_video/%(id)s.%(ext)s' # specifies output path and format
}

In [ ]:
# can be a list of links, in this case we are using a single playlist
playlist = ['https://youtube.com/playlist?list=PLX6CyFIhOy_z7cO9hBB8RhSw42CRI5jTe&si=bcdricJWnnIl8gRo']

In [ ]:
# initialize variable to hold each row of metadata
metadata = []
i = 1

for video in playlist:

  with YoutubeDL(options) as ydl:
    info_dict = ydl.extract_info(video)

    for entry in info_dict['entries']:
      row = []

      # change this according to language: en, fr, cn
      row.append("fr_" + str(i))
      row.append(entry['id'])
      row.append(entry['uploader'])
      row.append(entry['upload_date'])
      row.append(entry['title'])
      row.append(entry['duration'])
      row.append(entry['view_count'])
      row.append(entry['like_count'])
      row.append(entry['comment_count'])
      row.append(entry['original_url'])

      metadata.append(row)
      i += 1

In [ ]:
# turn metadata into a dataframe
headers = ['file_id', 'video_id', 'channel', 'date', 'title', 'duration_seconds', 'views', 'likes', 'comments', 'url']

metadata_df = pd.DataFrame(metadata,columns=headers)

In [ ]:
metadata_df.head()

,file_id,video_id,channel,date,title,duration_seconds,views,likes,comments,url
0,fr_1,w5rwVHtOjR0,QUB Radio,20240516,Richard a plein de belles idées pour le Villag...,37,4045,85,NaN,https://www.youtube.com/watch?v=w5rwVHtOjR0
1,fr_2,Jou0jd39ZgI,QUB Radio,20240423,Une police de l'anglais à l'école ? Benoit et ...,54,3401,49,NaN,https://www.youtube.com/watch?v=Jou0jd39ZgI
2,fr_3,ovJyh68t5sM,QUB Radio,20241205,La vérité sur les véhicules électriques!,44,5977,251,NaN,https://www.youtube.com/watch?v=ovJyh68t5sM
3,fr_4,IAvAu4O_8Uk,QUB Radio,20240213,"Hey le Parlement, vos états d'âmes et vos égos...",45,2796,92,NaN,https://www.youtube.com/watch?v=IAvAu4O_8Uk
4,fr_5,3D1zs_wJdFQ,QUB Radio,20240529,Dutrizac félicite le Hamas!,37,2528,125,NaN,https://www.youtube.com/watch?v=3D1zs_wJdFQ


In [ ]:
# create new annotations dataframe with the following columns
# file_id, title, speaker_gender, speaker_age, descriptive_tags, transcript
annotations_df = metadata_df.filter(['file_id', 'title'], axis=1)
annotations_df['speaker_gender'] = ' '
annotations_df['speaker_age'] = ' '
annotations_df['descriptive_tags'] = ' '
annotations_df['transcript'] = ' '

In [ ]:
annotations_df.head()

,file_id,title,speaker_gender,speaker_age,descriptive_tags,transcript
0,fr_1,Richard a plein de belles idées pour le Villag...,,,,
1,fr_2,Une police de l'anglais à l'école ? Benoit et ...,,,,
2,fr_3,La vérité sur les véhicules électriques!,,,,
3,fr_4,"Hey le Parlement, vos états d'âmes et vos égos...",,,,
4,fr_5,Dutrizac félicite le Hamas!,,,,


Here we specify the output filepath for the metadata spreadsheet.

In [ ]:
# file path for metadata - output excel sheet path
output = '/content/drive/My Drive/kickstarter_seed_project/fr_metadata.xlsx'

### Note!!!
The next three cell blocks were written to accommodate datasets over 100 videos that we want to keep in the same metadata sheet. Run this after extracting the additional videos, NOT when the first playlist is extracted.

In [ ]:
# this section for reading in the existing sheet to append the second half of the playlist
# since we can only download 100 videos at once
meta_df = pd.read_excel(output, sheet_name="metadata")
anno_df = pd.read_excel(output, sheet_name="annotations")

merge_meta_df = pd.concat([meta_df, metadata_df], ignore_index=True)
merge_anno_df = pd.concat([anno_df, annotations_df], ignore_index=True)

In [ ]:
merge_meta_df

In [ ]:
merge_anno_df

If there is only one playlist, or we are extracting the first playlist, run this block instead of the previous three.

In [ ]:
# if the playlist is less than 100 videos, run this block instead
merge_meta_df = metadata_df
merge_anno_df = annotations_df

In [ ]:
# create excel writer object to initialize new workbook
writer = pd.ExcelWriter(output, engine="xlsxwriter")

# write dataframes to different worksheets
merge_meta_df.to_excel(writer, sheet_name="metadata", index=False)
merge_anno_df.to_excel(writer, sheet_name="annotations", index=False)

# close the excel writer and output file
writer.close()

## Convert downloaded videos from mp4 to mp3

In [ ]:
!pip install moviepy

In [ ]:
from moviepy.editor import VideoFileClip

Here we define the video and audio folders for storing our files. Make these folders before running the rest of this code. If you are in Google Colab, refresh the notebook after creating the folders to catch the latest changes.

In [ ]:
# define folders where audio and video files are stored
video_folder = '/content/drive/My Drive/kickstarter_seed_project/fr_video'
audio_folder = '/content/drive/My Drive/kickstarter_seed_project/fr_audio'

Side note: The first two parts of this notebook (extracting and converting) are meant to be run in the same session. If returning to this notebook, run the code below to read the metadata file back as a dataframe.

In [ ]:
# only run this if you are converting the videos in a different session than you extracted
meta_fp = 'content/drive/My Drive/kickstarter_seed_project/en_metadata.xlsx'

metadata_df = pd.read_excel(meta_fp, sheet_name="metadata")

### Note!!
When running this section, make sure that only the videos we are renaming (and thus, extracting/converting) are in the folders defined above. If we have previously extracted other videos, move those to a subfolder or a different folder to prevent this code from breaking. If the folders only contain the videos to be renamed and it throws an error, check to see if videos were partially renamed and revert them back to their previous name (with ID instead of our established file convention).

TLDR: This code works if and only if all videos in `video_folder` have yet to be renamed and converted. You may have to refresh the notebook a few times to get it to work if it is still throwing indexing errors.

In [ ]:
for f in os.scandir(video_folder):
  if f.is_file():
    # first we rename the mp4 files using the file_id convention
    # instead of the default download id
    filename = os.path.basename(f)
    video_id = re.sub(r'.mp4', '', filename)
    file_id = metadata_df.loc[metadata_df['video_id'] == video_id, 'file_id'].iloc[0]
    new_fp = os.path.join(video_folder, file_id + ".mp4")
    os.rename(f, new_fp)

    # then we define the file names for the mp3 files
    mp3_file = file_id + ".mp3"
    mp3_output = os.path.join(audio_folder, mp3_file)

    # load the video file
    video = VideoFileClip(new_fp)

    # extract and save the audio file
    video.audio.write_audiofile(mp3_output)

## Extracting transcript from excel sheet to individual txt files
Future-proofing this project in case we need to extract transcripts to individual files. As before, `fp` defines the file path of the metadata file, and `ts_output` defines the output folder to hold all the transcript txts. If we're directly creating the transcripts as individual txt files, this section can be ignored.

In [ ]:
# file path of annotations/metadata file
fp = '/content/drive/My Drive/kickstarter_seed_project/en_metadata.xlsx'

In [ ]:
# get relevant columns from annotations sheet
transcripts_df = pd.read_excel(fp, sheet_name="annotations", usecols=['file_id', 'transcript'])

In [ ]:
transcripts_df.head()

In [ ]:
# output folder for individual txt files
ts_output = '/content/drive/My Drive/kickstarter_seed_project/transcripts'

In [ ]:
for index, row in transcripts_df.iterrows():
  filename = row['file_id'] + '.txt'
  filepath = os.path.join(ts_output, filename)

  with open(filename, 'w', encoding='utf-8') as txt_file:
      txt_file.write(row['transcript'])